In [10]:
import numpy as np
import pandas as pd

lambda_ = 1
d = 128

In [ ]:
def build_global_arm_dict(*arm_dicts):
    all_arms = set()
    for d in arm_dicts:
        all_arms.update(d.keys())

    global_arm_dict = {cid: i for i, cid in enumerate(sorted(all_arms))}
    return global_arm_dict

def init_global_Ab(global_arm_dict, d, lambda_reg=0.0):
    m = len(global_arm_dict)

    A_global = np.zeros((m, d, d), dtype=np.float32)
    b_global = np.zeros((m, d), dtype=np.float32)

    if lambda_reg > 0:
        for i in range(m):
            A_global[i] = lambda_reg * np.eye(d)

    return A_global, b_global

def merge_one_day(
    A_day,
    b_day,
    arm_dict_day,
    A_global,
    b_global,
    global_arm_dict
):
    for cid, local_idx in arm_dict_day.items():
        global_idx = global_arm_dict[cid]

        A_global[global_idx] += A_day[local_idx]
        b_global[global_idx] += b_day[local_idx]

def merge_days(days_data, d):
    """
    days_data: list of tuples
        [(A_day, b_day, arm_dict_day), ...]
    """

    arm_dicts = [arm_dict for _, _, arm_dict in days_data]
    global_arm_dict = build_global_arm_dict(*arm_dicts)

    A_global, b_global = init_global_Ab(global_arm_dict, d)

    for A_day, b_day, arm_dict_day in days_data:
        merge_one_day(
            A_day, b_day, arm_dict_day,
            A_global, b_global, global_arm_dict
        )

    return A_global, b_global, global_arm_dict

In [11]:
A_01 = np.load('disjoint_neural_linucb/disjoined_neural_linUCB_offline_matrices_2026-02-01_1770031258/A_47000.npy')
A_02 = np.load('disjoint_neural_linucb/disjoined_neural_linUCB_offline_matrices_2026-02-02_1770115825/A_40000.npy')

arm_dict_01 = pd.read_pickle('disjoint_neural_linucb/disjoined_neural_linUCB_offline_matrices_2026-02-01_1770031258/arm_index_47000.pkl')
arm_dict_02 = pd.read_pickle('disjoint_neural_linucb/disjoined_neural_linUCB_offline_matrices_2026-02-02_1770115825/arm_index_40000.pkl')

In [12]:
A_01.shape, A_02.shape

((8308, 128, 128), (8311, 128, 128))

In [18]:
A_01[-1][0][0]

1.0043239309452474

In [16]:
A_01_ = A_01 - lambda_ * np.eye(d, dtype=np.float32)

In [19]:
A_01_[-1][0][0]

0.004323930945247412

In [2]:
A_05 = np.load('tpfy/neural_linUCB_offline_matrices_2026-02-05_1770375089/A_39800.npy')
A_06 = np.load('tpfy/neural_linUCB_offline_matrices_2026-02-06_1770463379/A_42600.npy')
A_07 = np.load('tpfy/neural_linUCB_offline_matrices_2026-02-07_1770548252/A_40100.npy')
A_08 = np.load('tpfy/neural_linUCB_offline_matrices_2026-02-08_1770636411/A_49300.npy')
A_09 = np.load('tpfy/neural_linUCB_offline_matrices_2026-02-09_1770723470/A_39300.npy')
A_10 = np.load('tpfy/neural_linUCB_offline_matrices_2026-02-10_1770809120/A_38400.npy')
A_11 = np.load('tpfy/neural_linUCB_offline_matrices_2026-02-11_1770893443/A_30200.npy')

In [7]:
lambda_ = 1
d = 128
gamma = 0.95
regularized_matrix = lambda_ * np.eye(d, dtype=np.float32)

In [4]:
# reseted matrices save
np.save('progressive_matrices/reseted/A_2026-02-05.npy', A_05 + regularized_matrix)
np.save('progressive_matrices/reseted/A_2026-02-06.npy', A_06 + regularized_matrix)
np.save('progressive_matrices/reseted/A_2026-02-07.npy', A_07 + regularized_matrix)
np.save('progressive_matrices/reseted/A_2026-02-08.npy', A_08 + regularized_matrix)
np.save('progressive_matrices/reseted/A_2026-02-09.npy', A_09 + regularized_matrix)
np.save('progressive_matrices/reseted/A_2026-02-10.npy', A_10 + regularized_matrix)
np.save('progressive_matrices/reseted/A_2026-02-11.npy', A_11 + regularized_matrix)

In [8]:
# cumulative matrices save
np.save('progressive_matrices/cumulative/A_2026-02-05.npy', regularized_matrix + A_05 )
np.save('progressive_matrices/cumulative/A_2026-02-06.npy', regularized_matrix + A_05 + A_06 )
np.save('progressive_matrices/cumulative/A_2026-02-07.npy', regularized_matrix + A_05 + A_06 + A_07 )
np.save('progressive_matrices/cumulative/A_2026-02-08.npy', regularized_matrix + A_05 + A_06 + A_07 + A_08 )
np.save('progressive_matrices/cumulative/A_2026-02-09.npy', regularized_matrix + A_05 + A_06 + A_07 + A_08 + A_09 )
np.save('progressive_matrices/cumulative/A_2026-02-10.npy', regularized_matrix + A_05 + A_06 + A_07 + A_08 + A_09 + A_10 )
np.save('progressive_matrices/cumulative/A_2026-02-11.npy', regularized_matrix + A_05 + A_06 + A_07 + A_08 + A_09 + A_10 + A_11 )

In [9]:
# cumulative matrices save
np.save('progressive_matrices/penalized/A_2026-02-05.npy', regularized_matrix + A_05 )
np.save('progressive_matrices/penalized/A_2026-02-06.npy', gamma * (regularized_matrix + A_05) + A_06 )
np.save('progressive_matrices/penalized/A_2026-02-07.npy', (gamma ** 2) * (regularized_matrix + A_05) + ((gamma**1) * A_06) + A_07 )
np.save('progressive_matrices/penalized/A_2026-02-08.npy', (gamma ** 3) * (regularized_matrix + A_05) + ((gamma**2) * A_06) + ((gamma**1) * A_07) + A_08 )
np.save('progressive_matrices/penalized/A_2026-02-09.npy', (gamma ** 4) * (regularized_matrix + A_05) + ((gamma**3) * A_06) + ((gamma**2) * A_07) + ((gamma ** 1) * A_08) + A_09)
np.save('progressive_matrices/penalized/A_2026-02-10.npy', (gamma ** 5) * (regularized_matrix + A_05) + ((gamma**4) * A_06) + ((gamma**3) * A_07) + ((gamma ** 2) * A_08) + ((gamma ** 1) * A_09) + A_10)
np.save('progressive_matrices/penalized/A_2026-02-11.npy', (gamma ** 6) * (regularized_matrix + A_05) + ((gamma**5) * A_06) + ((gamma**4) * A_07) + ((gamma ** 3) * A_08) + ((gamma ** 2) * A_09) + ((gamma ** 1) * A_10) + A_11)

In [20]:
import numpy as np

In [23]:
A = np.load('test/A.npy')
A_inv = np.load('test/A_inv.npy')

In [24]:
A_inv

array([[ 1.87200680e-02,  1.14618265e-03, -5.22993319e-03, ...,
         7.24113896e-04, -1.56477117e-03, -2.52741319e-03],
       [ 1.14618265e-03,  4.03725095e-02, -1.10616640e-03, ...,
         2.76063127e-03, -4.28850217e-05,  3.83051171e-04],
       [-5.22993319e-03, -1.10616640e-03,  9.04234871e-02, ...,
         6.39436767e-03,  5.90717944e-04, -2.41369056e-03],
       ...,
       [ 7.24113896e-04,  2.76063127e-03,  6.39436767e-03, ...,
         5.51748872e-02,  1.03154154e-04,  2.61685136e-03],
       [-1.56477117e-03, -4.28850217e-05,  5.90717944e-04, ...,
         1.03154154e-04,  2.51352247e-02, -1.99194555e-03],
       [-2.52741319e-03,  3.83051171e-04, -2.41369056e-03, ...,
         2.61685136e-03, -1.99194555e-03,  4.35083807e-02]], dtype=float32)